In [1]:
from cloudmanufacturing.data import read_fatahi_dataset
from cloudmanufacturing.solvers.mip_solver import mip_solve
from cloudmanufacturing.validation import construct_delta, objvalue

from joblib import Parallel, delayed
from tqdm import trange
import pandas as pd
import numpy as np
import random
import openpyxl
from cloudmanufacturing.data_generation import create_excel_table
random.seed(42)

c:\Users\dmitrii\miniconda3\envs\graph\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Fatahi

In [9]:
dataset = read_fatahi_dataset("../data/fatahi.xlsx")

100%|██████████| 18/18 [00:00<00:00, 244.52it/s]


In [10]:
def fatahi_problem(i, problem):
    print(problem['n_operations'])
    print(problem['n_suboperations'])
    print(problem['n_cities'])
    delta, gamma, status, value = mip_solve(problem)

    if status.name == 'OPTIMAL':
        np.save(f'./fatahi_solutions/delta_{i}_op.npy', delta)
        np.save(f'./fatahi_solutions/gamma_{i}_op.npy', gamma)
        return {'i':i,
                'value': value,
                'objvalue': objvalue(problem, gamma, delta),
                'status': status.name}
    if status.name == 'FEASIBLE':
        np.save(f'./fatahi_solutions/delta_{i}_fb.npy', delta)
        np.save(f'./fatahi_solutions/gamma_{i}_fb.npy', gamma)
        return {'i':i,
                'value': value,
                'objvalue': objvalue(problem, gamma, delta),
                'status': status.name}
    print(i," wasn't solved")
    return None

In [11]:
sum_info = Parallel(n_jobs=-1)(
    delayed(fatahi_problem)(i, dataset[i]) for i in trange(len(dataset))
)
sum_info = [info for info in sum_info if info is not None]

100%|██████████| 18/18 [00:00<?, ?it/s]


In [12]:
pd.DataFrame(sum_info)

,i,value,objvalue,status
0,0,4030.331095,4030.331095,OPTIMAL
1,1,5560.953381,5560.953381,OPTIMAL
2,2,5794.687894,5794.687894,OPTIMAL
3,3,9691.854167,9691.854167,OPTIMAL
4,4,10895.243418,10895.243418,OPTIMAL
5,5,10866.234192,10866.234192,OPTIMAL
6,6,3304.323814,3304.323814,FEASIBLE
7,7,4208.375428,4208.375428,OPTIMAL
8,8,4701.340722,4701.340722,FEASIBLE
9,9,10390.543328,10390.543328,FEASIBLE


### Train dataset

In [4]:
dataset_train = read_fatahi_dataset("../data/train_data.xlsx")

100%|██████████| 100/100 [00:10<00:00,  9.13it/s]


In [7]:
def train_problem(i, problem):
    delta, gamma, status, value = mip_solve(problem)

    if status.name == 'OPTIMAL':
        np.save(f'./train_solutions/delta_{i}_op.npy', delta)
        np.save(f'./train_solutions/gamma_{i}_op.npy', gamma)
        return {'i':i,
                'value': value,
                'objvalue': objvalue(problem, gamma, delta),
                'status': status.name}
    if status.name == 'FEASIBLE':
        np.save(f'./train_solutions/delta_{i}_fb.npy', delta)
        np.save(f'./train_solutions/gamma_{i}_fb.npy', gamma)
        return {'i':i,
                'value': value,
                'objvalue': objvalue(problem, gamma, delta),
                'status': status.name}
    print(i," wasn't solved")
    return None

In [8]:
sum_info_train = Parallel(n_jobs=-1)(
    delayed(train_problem)(i, dataset_train[i]) for i in trange(len(dataset_train))
)
sum_info_train = [info for info in sum_info_train if info is not None]

100%|██████████| 100/100 [04:41<00:00,  2.81s/it]


#### Data generation

In [2]:
N = 100
stop_count = 100

In [3]:
file_path = '../data/train_data.xlsx'

trios = []
for i in range(N):
    trios.append(tuple((random.randint(5,10),random.randint(10,15),random.randint(10,15))))
trios.sort()

wb = openpyxl.Workbook()
del wb['Sheet']

for n_operations, n_suboperations, n_cities in trios:
    wb = create_excel_table(wb, n_operations, n_suboperations, n_cities, n_problem=1)

wb.save(file_path)

### junk

In [ ]:
# sum_info = []
# for i in trange(len(dataset)):
#     problem = dataset[i]
#     delta, gamma, status, value = mip_solve(problem)
#     if status.name == 'OPTIMAL':
#         np.save(f'./mip_solutions/delta_{i}_opt.npy', delta)
#         np.save(f'./mip_solutions/gamma_{i}_opt.npy', gamma)
#         sum_info.append({'value':value,
#                         'objvalue':objvalue(problem, gamma, delta),
#                         'status':status
#                         })
#     if status.name == 'FEASIBLE':
#         np.save(f'./mip_solutions/delta_{i}_opt.npy', delta)
#         np.save(f'./mip_solutions/gamma_{i}_opt.npy', gamma)
#         sum_info.append({'value':value,
#                         'objvalue':objvalue(problem, gamma, delta),
#                         'status':status
#                         })
# pd.DataFrame(sum_info).to_csv('./mip_solutions/info.csv',index=False)